In [1]:
import urllib.request
from urllib.request import (urlopen, urlparse, urlretrieve)

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

import time
import os

In [2]:
chrome_path = '/home/jwjang/chromedriver'
base_url = "https://www.google.co.kr/imghp"

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

driver = webdriver.Chrome(chrome_path, chrome_options=options)

/tmp/ipykernel_12133/3548743907.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path, chrome_options=options)
/tmp/ipykernel_12133/3548743907.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_path, chrome_options=options)


In [4]:
def selenium_scroll_option() :
    SCROLL_PAUSE_SEC = 1

    # 스크롤 높이 가져옴
    last_height = driver.execute_script(
        "return document.body.scrollHeight")

    while True :
        # 끝까지 스크롤 다운
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script(
            "return document.body.scrollHeight")
        if new_height == last_height :
            break
        last_height = new_height


item_list = ['코카콜라 캔']
for i in range(len(item_list)):
    image_name = item_list[i]
    # driver = webdriver.Chrome(chrome_path)
    driver.get("http://www.google.co.kr/imghp?hl=ko")
    browser = driver.find_element(By.NAME, 'q')
    browser.send_keys(item_list[i])
    browser.send_keys(Keys.RETURN)

    selenium_scroll_option()# 스크롤 하여 이미지 확보
    driver.find_element(By.XPATH,
        '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()
    selenium_scroll_option()

    # 이미지 저장 src 요소를 리스트업 해서 이미지 url 저장
    image = driver.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
    # 클래스 네임에서 공백은 . 을 찍어줌
    # print(image)
    image_url = []
    for i in image:
        if i.get_attribute("src") != None :
            image_url.append(i.get_attribute("src"))
        else :
            image_url.append(i.get_attribute("data-src"))

    #전체 이미지 개수
    print(f"전체 다운로드한 이미지 개수 : {len(image_url)}")
    image_url = pd.DataFrame(image_url)[0].unique()

    for i in range(len(item_list)):
        # 해당하는 파일에 이미지 다운로드
        os.makedirs(f"./image{i}", exist_ok=True)
        new_img = f"./image{i}/"
        if image_name == f'{item_list[i]}':
            for t, url in enumerate(image_url, 0):
                urlretrieve(url, new_img + image_name + "_" + str(t) + ".png")
            driver.close()
    print("완료")

전체 다운로드한 이미지 개수 : 589
완료
